In [ ]:
!pip uninstall -y sentence-transformers

In [82]:
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 489.6 MB 48 kB/s s eta 0:00:014
  Attempting uninstall: tensorflow-gpu
    Found existing installation: tensorflow-gpu 2.3.0
    Uninstalling tensorflow-gpu-2.3.0:
      Successfully uninstalled tensorflow-gpu-2.3.0
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 108 kB 1.6 MB/s eta 0:00:01


In [6]:
import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm

In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

NameError: name 'hub' is not defined

In [2]:
papers_features = pd.read_csv("processed_data/SSORC_CS_2010_2021_papers_features.csv", index_col = 0)

/home/user/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [40]:
sorted(papers_features["magId"].values)

False

In [4]:
papers_edges = pd.read_csv("processed_data/SSORC_CS_2010_2021_papers_edge_list.csv", index_col = 0)

In [5]:
authors_edges = pd.read_csv("processed_data/SSORC_CS_2010_2021_authors_edge_list.csv", index_col = 0)

In [6]:
authors_papers = pd.read_csv("processed_data/SSORC_CS_2010_2021_authors_edges_papers.csv", index_col = 0)

In [7]:
papers_id = papers_features["id"]
id_to_index_id = {papers_id[i]:i for i in tqdm(range(len(papers_id)))}

100%|██████████| 2682709/2682709 [00:26<00:00, 100305.03it/s]


In [10]:
authors_papers_unzipped = authors_papers["papers_ids"].map(lambda x: x.replace("[", "").replace("]", "").replace("'", "").split(", "))

In [17]:
authors_papers_indexed = [[id_to_index_id[authors_papers_unzipped[i][j]] for j in range(len(authors_papers_unzipped[i]))] for i in tqdm(range(len(authors_papers_unzipped)))]



  0%|          | 0/30796749 [00:00<?, ?it/s]

  0%|          | 3352/30796749 [00:00<15:18, 33512.84it/s]

  0%|          | 6704/30796749 [00:00<17:26, 29414.98it/s]

  0%|          | 9678/30796749 [00:00<18:51, 27203.60it/s]

  0%|          | 12423/30796749 [00:00<19:53, 25791.56it/s]

  0%|          | 15017/30796749 [00:00<20:12, 25395.92it/s]

  0%|          | 18127/30796749 [00:00<18:52, 27186.19it/s]

  0%|          | 20865/30796749 [00:00<18:58, 27021.61it/s]

  0%|          | 23580/30796749 [00:00<19:27, 26356.67it/s]

  0%|          | 26344/30796749 [00:00<19:10, 26738.17it/s]

  0%|          | 29027/30796749 [00:01<19:19, 26524.18it/s]

  0%|          | 31757/30796749 [00:01<19:09, 26753.46it/s]

  0%|          | 34545/30796749 [00:01<18:56, 27070.87it/s]

  0%|          | 37256/30796749 [00:01<19:58, 25666.31it/s]

  0%|          | 39839/30796749 [00:01<20:14, 25322.45it/s]

  0%|          | 42439/30796749 [00:01<20:05, 25517.02it/s]

  0%|          | 45000/30796749 [00:01<2

In [22]:
authors_papers_indexed_str = [str(authors_papers_indexed[i]) for i in tqdm(range(len(authors_papers_indexed)))]

100%|██████████| 30796749/30796749 [00:21<00:00, 1464775.17it/s]


In [24]:
pd.DataFrame(authors_papers_indexed_str, columns = ["papers_indices"]).to_csv("processed_data/SSORC_CS_2010_2021_authors_edges_papers_indices.csv")

In [30]:
df = papers_features[papers_features[['id', 'title', 'paperAbstract', 'year', 'journalName', 'fieldsOfStudy']].notna()]

In [151]:
papers_features_abstracts = list(papers_features["paperAbstract"])

In [148]:
papers_features["paperAbstract"] = papers_features["paperAbstract"].fillna("No abstract provided")

In [152]:
vectorized_abstracts = []
for i in tqdm(range(len(papers_features_abstracts))):
    abstract = papers_features_abstracts[i]
    vectorized_abstracts.append(model([abstract])[0])

100%|██████████| 2682709/2682709 [1:50:02<00:00, 406.33it/s]  


In [167]:
vectorized_abstracts_list = [vectorized_abstracts[i].numpy() for i in tqdm(range(len(vectorized_abstracts)))]

100%|██████████| 2682709/2682709 [00:08<00:00, 329240.28it/s]


In [174]:
vectorized_abstracts_df = pd.DataFrame(vectorized_abstracts_list)

In [188]:
pd.DataFrame(pca_result).to_csv("processed_data/SSORC_CS_2010_2021_papers_features_vectorized_compressed_32.csv")

In [184]:
from sklearn.decomposition import PCA

pca = PCA(n_components=32)
pca_result = pca.fit_transform(X)

In [193]:
papers_edge_list_indexed = papers_edges.values
for i in tqdm(range(len(papers_edge_list_indexed))):
    pair = papers_edge_list_indexed[i]
    for j in range(len(pair)):
        pair[j] = id_to_index_id[pair[j]]

100%|██████████| 17921409/17921409 [00:27<00:00, 653417.67it/s]


In [194]:
papers_edge_list_indexed_np = pd.DataFrame(papers_edge_list_indexed)

In [200]:
papers_edge_list_indexed_np.to_csv("processed_data/SSORC_CS_2010_2021_papers_edge_list_indexed.csv")

### Execute after subdataset extraction
Journals of papers.

In [8]:
dataset_name = 'SSORC_CS_10_21_419_2055_primus' #'SSORC_CS_10_21_3340_16830_115907_primus'

In [20]:
citation_graph = nx.read_edgelist("datasets/" + dataset_name + "/" + dataset_name + "_" + "papers.edgelist", create_using = nx.DiGraph)

nodes = list(citation_graph.nodes())
nodes = [int(nodes[i]) for i in range(len(nodes))]

papers_features.loc[nodes, :]["journalName"].to_frame().to_csv("journals.csv")